In [ ]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  3717k      0 --:--:-- --:--:-- --:--:-- 3731k


In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [ ]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [ ]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0030, 0.0149, 0.0045,  ..., 0.0316, 0.0093, 0.0484],
        [0.0391, 0.0295, 0.0077,  ..., 0.0061, 0.0093, 0.0233],
        [0.0094, 0.0131, 0.0115,  ..., 0.0347, 0.0217, 0.0316],
        ...,
        [0.0418, 0.0289, 0.0364,  ..., 0.0296, 0.0182, 0.0390],
        [0.0333, 0.0219, 0.0277,  ..., 0.0402, 0.0059, 0.0311],
        [0.0060, 0.0420, 0.0384,  ..., 0.0252, 0.0261, 0.0160]])
item_factors.weight tensor([[0.0339, 0.0125, 0.0496,  ..., 0.0038, 0.0500, 0.0406],
        [0.0438, 0.0302, 0.0096,  ..., 0.0058, 0.0257, 0.0467],
        [0.0105, 0.0372, 0.0088,  ..., 0.0482, 0.0272, 0.0408],
        ...,
        [0.0173, 0.0326, 0.0255,  ..., 0.0412, 0.0301, 0.0379],
        [0.0374, 0.0421, 0.0254,  ..., 0.0204, 0.0251, 0.0022],
        [0.0212, 0.0367, 0.0220,  ..., 0.0069, 0.0263, 0.0371]])


In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


iter #0 Loss: 11.064539746584625
iter #1 Loss: 4.749444664734875
iter #2 Loss: 2.4776223112786484
iter #3 Loss: 1.72177541634153
iter #4 Loss: 1.3460204232919035
iter #5 Loss: 1.1284338527827094
iter #6 Loss: 0.9915084930392086
iter #7 Loss: 0.9002359872542057
iter #8 Loss: 0.8372054895395555
iter #9 Loss: 0.7921712103909647
iter #10 Loss: 0.7592169020306035
iter #11 Loss: 0.734483423155879
iter #12 Loss: 0.7160191048825453
iter #13 Loss: 0.7014374917912
iter #14 Loss: 0.690310317020731
iter #15 Loss: 0.6816033662348835
iter #16 Loss: 0.6750705706589113
iter #17 Loss: 0.669739288727039
iter #18 Loss: 0.6658609225816533
iter #19 Loss: 0.6628223651935001
iter #20 Loss: 0.6604955623370742
iter #21 Loss: 0.6587029967059944
iter #22 Loss: 0.6573294878459824
iter #23 Loss: 0.6564567382565609
iter #24 Loss: 0.6555627360183576
iter #25 Loss: 0.6549196255388599
iter #26 Loss: 0.6539314403978701
iter #27 Loss: 0.6528688056517373
iter #28 Loss: 0.6521614116006696
iter #29 Loss: 0.6506869314890827

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 0.6636,  1.7670,  1.5851,  ...,  1.5591,  0.9846,  1.3279],
        [ 0.2886,  0.8389,  0.4740,  ...,  0.0211,  1.7432,  1.8250],
        [-0.1276,  0.3505, -1.8636,  ..., -1.8244,  2.3191,  2.1920],
        ...,
        [ 1.5623,  0.0149,  0.6837,  ...,  2.2128,  1.5504, -0.0838],
        [ 0.8281,  0.7045,  0.6922,  ...,  1.0350,  0.5839,  1.2660],
        [ 0.8088,  1.1780,  0.7822,  ...,  0.9027,  2.0788,  1.3200]],
       device='cuda:0')
item_factors.weight tensor([[ 0.4925,  0.3693,  0.7865,  ...,  0.1419,  0.4939,  0.3609],
        [ 0.1813,  0.2992,  0.4251,  ...,  0.5503, -0.0829,  0.6948],
        [ 0.5640,  0.3030,  0.5318,  ...,  0.5683,  0.7503,  0.3950],
        ...,
        [ 0.3246,  0.3380,  0.3319,  ...,  0.3477,  0.3367,  0.3458],
        [ 0.3952,  0.4012,  0.3834,  ...,  0.3810,  0.3876,  0.3640],
        [ 0.4014,  0.4191,  0.4042,  ...,  0.3881,  0.4108,  0.4209]],
       device='cuda:0')


In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Terminator 2: Judgment Day (1991)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Usual Suspects, The (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
Cluster #1
	 Godzilla (1998)
	 Super Mario Bros. (1993)
	 Honey, I Blew Up the Kid (1992)
	 Battlefield Earth (2000)
	 Mighty Morphin Power Rangers: The Movie (1995)
	 Superman IV: The Quest for Peace (1987)
	 Next Karate Kid, The (1994)
	 Volcano (1997)
	 Karate Kid, Part III, The (1989)
	 Rambo III (1988)
Cluster #2
	 Alien (1979)
	 Die Hard (1988)
	 Groundhog Day (1993)
	 Terminator, The (1984)
	 Aliens (1986)
	 Austin Powers: The Spy Who Shagged Me (1999)
	 Clerks (1994)
	 American Pie (1999)
	 Heat (1995)
	 Austin Powers: International Man of Mystery (1997)
C